# P-multigrid Example

In [ ]:
# dependencies
using LFAToolkit
using LinearAlgebra
using Plots

In [ ]:
# setup
dimension = 2
finep = 3
coarsep = 5
mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end
finebasis = TensorH1LagrangeBasis(finep, finep+1, dimension)
coarsebasis = TensorH1LagrangeBasis(coarsep, finep+1, dimension)
lagrangequadrature = true
ctofbasis = TensorH1LagrangeBasis(coarsep, finep, dimension, lagrangequadrature)

function diffusionweakform(du::Array{Float64}, w::Array{Float64})
    dv = du*w[1]
    return [dv]
end

# fine grid diffusion operator
fineinputs = [
    OperatorField(finebasis, [EvaluationMode.gradient]),
    OperatorField(finebasis, [EvaluationMode.quadratureweights]),
]
fineoutputs = [OperatorField(finebasis, [EvaluationMode.gradient])]
finediffusion = Operator(diffusionweakform, mesh, fineinputs, fineoutputs)

# coarse grid diffusion operator
coarseinputs = [
    OperatorField(coarsebasis, [EvaluationMode.gradient]),
    OperatorField(coarsebasis, [EvaluationMode.quadratureweights]),
]
coarseoutputs = [OperatorField(coarsebasis, [EvaluationMode.gradient])]
coarsediffusion = Operator(diffusionweakform, mesh, coarseinputs, coarseoutputs)

# Jacobi smoother
jacobi = Jacobi(finediffusion)

# p-multigrid preconditioner
multigrid = PMultigrid(finediffusion, coarsediffusion, jacobi, [ctofbasis])

In [ ]:
# compute operator symbols
numberruns = 100

# compute and plot smoothing factor
if dimension == 1
    # compute
    maxeigenvalues = zeros(numberruns)
    for i in 1:numberruns
        A = computesymbols(multigrid, [0.838], [1, 1], [π/finep+((2finep-1)π/finep-π/finep)*(i/numberruns)])
        eigenvalues = [abs(val) for val in eigvals(A)]
        maxeigenvalues[i] = max(eigenvalues...)
    end

    # plot
    plot(maxeigenvalues, title="P-Multigrid Symbols")
elseif dimension == 2
    # compute
    maxeigenvalues = zeros(numberruns, numberruns)
    for i in 1:numberruns, j in 1:numberruns
        A = computesymbols(multigrid, [0.838], [1, 1], [π/finep+((2finep-1)π/finep-π/finep)*(i/numberruns), π/finep+((2finep-1)π/finep-π/finep)*(j/numberruns)])
        eigenvalues = [abs(val) for val in eigvals(A)]
        maxeigenvalues[i, j] = max(eigenvalues...)
    end

    # plot
    heatmap(maxeigenvalues, title="P-Multigrid Symbols", transpose=true)
end